### 1.洗床数据清洗与整合

In [1]:
import pandas as pd
import re
import calendar
import os
from time import sleep
from tqdm.auto import tqdm

# 初始化文件
if os.path.exists('RegenerationRawInfo.csv'):
    os.remove('RegenerationRawInfo.csv')

valid_columns = [2,5,8]
years = [2019,2020,2021,2022]

regeneration_df = pd.DataFrame({'date':[], 'regeneration':[]})
regeneration_df.to_csv("RegenerationRawInfo.csv",mode='a',index=False)


for year in tqdm(years,desc="year"):


    # 遍历路径文件
    for month in tqdm(range(1,13,1),desc="month"):
        file_name = '干熄焦生产记录台账/{0}/干熄焦生产记录台账（{1}月）.xlsx'.format(year,month)
        #file_date_info = re.findall(r'\d{1,}', file_name) # 提取路径文件信息(年、月）
        #month = file_date_info[1]
        # year = file_date_info[0]

        # 返回（当月总周数，当月总天数）
        month_range = calendar.monthrange(int(year),int(month))
        
        # 遍历表单
        for day in range(1,month_range[1]+1,1):  # 从第1天开始至当月最后一天
            try:
                regeneration_date = pd.read_excel(file_name, sheet_name = "{}日".format(day), header=None) # 读取当日表单
            except FileNotFoundError as file_error:
                print(file_error)
                break
                
            if (year < 2020 or ( year==2020 and month <= 9)):
                valid_index1 = 25
                valid_index2 = 26
            else :
                valid_index1 = 27
                valid_index2 = 28
                
            regeneration_cell = regeneration_date.iloc[valid_index1:valid_index2,[2,5,8]].astype(str) # 只保留包含洗床单元格的数据
            
            # 构建日期DataFrame
            regenerate_date = {'date':["{0}-{1}-{2}".format(year, month, day)]} # 再生日期信息
            regeneration_date_df = pd.DataFrame(data=regenerate_date)
        
            # 对单元格遍历，拆分、分割
            for valid_column in valid_columns:
                regeneration_split_cell = regeneration_cell[valid_column].T.str.split(r'\s+', expand=True) # 转置,（以至少1个空格)分割
            
                # 根据拆分的单元格长度，遍历追加入文件
                for l in range(int(regeneration_split_cell.size)):
                    regeneration_split_cell[l] =  regeneration_split_cell[l].astype(str)
                        
                    # 重建洗床信息DataFrame
                    regeneration_split_cell_value = regeneration_split_cell[l].values
                    regeneration_split_cell_value_df = pd.DataFrame(regeneration_split_cell_value)
                    frames = [regeneration_date_df, regeneration_split_cell_value_df]
                    result = pd.concat(frames,join="outer",axis=1, ignore_index=True)
                    result.to_csv("RegenerationRawInfo.csv",mode='a',header=False, index=False)

# 剔除空行
regeneration_raw_info = pd.read_csv("RegenerationRawInfo.csv")
regeneration_raw_info.dropna(axis='rows', inplace=True)
regeneration_raw_info.reset_index(drop=True, inplace=True)  # drop=True能避免把旧列插入重排列

regeneration_raw_info.to_csv("RegenerationRawInfo.csv",mode='w', index=False)

year:   0%|          | 0/4 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（12月）.xlsx'


### 2.洗床数据分割

In [2]:
rg_info = pd.read_csv("RegenerationRawInfo.csv")
pd.to_datetime(rg_info.date)

rg_split = rg_info['regeneration'].str.split(r' ?分? ?再生|，|进|置换', regex=True, expand=True)

rg_split['time'] = rg_split[0]
rg_split['cation_resin'] = rg_split[1]

rg_split_draft = pd.concat([rg_info.date, rg_split['time'], rg_split['cation_resin']], axis=1)
rg_split_draft.replace("：",":", inplace=True)
rg_split_draft.to_csv("RegenerationSplitInfo.csv",index=False)

In [3]:
rg_draft = pd.read_csv("RegenerationSplitInfo.csv")

1#阳床

In [4]:
cation_resin_1 = rg_draft.loc[rg_draft.iloc[:,2].str.contains(r'1#阳床',regex=True, na=False)]

# 规范数据格式
cation_resin_1 = cation_resin_1[cation_resin_1['cation_resin'].notnull()].copy()
cation_resin_1.replace(r'：',":", regex=True, inplace=True)
cation_resin_1.replace(r';',":", regex=True, inplace=True)
cation_resin_1.replace(r'点',":", regex=True, inplace=True)

# 统一时间格式
cation_resin_1['time'] = pd.to_datetime(cation_resin_1['date'] +" "+ cation_resin_1['time'])

# 洗床间隔时间
cation_resin_1['time_diff'] = pd.to_datetime(cation_resin_1['time']).diff()

cation_resin_1.to_csv("cation_resin_1.csv",index=False)

2#阳床

In [5]:
cation_resin_2 = rg_draft.loc[rg_draft.iloc[:,2].str.contains(r'2#阳',regex=True, na=False)]

# 规范数据格式
cation_resin_2 = cation_resin_2[cation_resin_2['cation_resin'].notnull()].copy()
cation_resin_2.replace(r'：',":", regex=True, inplace=True)
cation_resin_2.replace(r';',":", regex=True, inplace=True)
cation_resin_2.replace(r'点',":", regex=True, inplace=True)
cation_resin_2.replace(r'\.',":", regex=True, inplace=True)
cation_resin_2.replace(r'::',":", regex=True, inplace=True)

# 时间标准化
cation_resin_2['time'] = pd.to_datetime(cation_resin_2['date'] +" "+ cation_resin_2['time'])

# 洗床间隔时间
cation_resin_2['time_diff'] = pd.to_datetime(cation_resin_2['time']).diff()

cation_resin_2.to_csv("cation_resin_2.csv",index=False)

3#阳床

In [6]:
cation_resin_3 = rg_draft.loc[rg_draft.iloc[:,2].str.contains(r'3#阳',regex=True, na=False)]

# 规范数据格式
cation_resin_3 = cation_resin_3[cation_resin_3['cation_resin'].notnull()].copy()
cation_resin_3.replace(r'：',":", regex=True, inplace=True)
cation_resin_3.replace(r';',":", regex=True, inplace=True)
cation_resin_3.replace(r'点',":", regex=True, inplace=True)
cation_resin_3.replace(r'\.',":", regex=True, inplace=True)
cation_resin_3.replace(r'::',":", regex=True, inplace=True)

# 时间标准化
cation_resin_3['time'] = pd.to_datetime(cation_resin_3['date'] +" "+ cation_resin_3['time'])

# 洗床间隔时间
cation_resin_3['time_diff'] = pd.to_datetime(cation_resin_3['time']).diff()

cation_resin_3.to_csv("cation_resin_3.csv",index=False)

### 3.异常数据处理

2021-01-14 中间间隔143天40小时系1#阳床树脂泄漏维护，故剔除该行。

In [10]:
cation_resin_1.drop([cation_resin_1['time_diff'].idxmax()], axis=0, inplace=True)

#### 1#阳床异常数据处理：
1.删除使用时长大于72h，小于8h的数据。（床体维护）

2.大于40h的数据乘以0.8备用系数。（床体备用）

In [11]:
# 阳床间隔单位为小时
cation_resin_1['time_diff'] = (cation_resin_1['time_diff'].dt.total_seconds() / 60 / 60 ).round(2)
cation_resin_2['time_diff'] = (cation_resin_2['time_diff'].dt.total_seconds() / 60 / 60 ).round(2)
cation_resin_3['time_diff'] = (cation_resin_3['time_diff'].dt.total_seconds() / 60 / 60 ).round(2)

In [12]:
cation_resin_1.describe()

,time_diff
count,857.000000
mean,36.015589
std,19.787502
min,3.480000
25%,22.150000
50%,33.580000
75%,44.400000
max,170.170000


In [13]:
authentic_cs_1 = cation_resin_1.loc[(cation_resin_1['time_diff'] < 50) & (cation_resin_1['time_diff'] > 6)]
# (authentic_cs_1['time_diff'].loc[(authentic_cs_1['time_diff'] > 50 )])*0.8
# print(authentic_cs_1.describe())
# cation_resin_1 = cs_1_authentic.copy()
# authentic_cs_1_df = cation_resin_1.sort_values("date", ascending=False)
authentic_cs_1.to_csv("authentic_cs_1.csv", index=False)

In [14]:
authentic_cs_1.describe()

,time_diff
count,698.000000
mean,29.392880
std,10.794571
min,6.500000
25%,20.830000
50%,29.985000
75%,38.000000
max,49.950000


In [15]:
authentic_cs_2 = cation_resin_2.loc[(cation_resin_2['time_diff'] < 50) & (cation_resin_2['time_diff'] > 6)]
authentic_cs_2.to_csv("authentic_cs_2.csv", index=False)

In [16]:
cation_resin_2.describe()

,time_diff
count,903.000000
mean,37.921406
std,27.328775
min,3.270000
25%,24.410000
50%,33.450000
75%,43.750000
max,405.330000


In [17]:
authentic_cs_3 = cation_resin_3.loc[(cation_resin_3['time_diff'] < 50) & (cation_resin_3['time_diff'] > 6)]
authentic_cs_3.to_csv("authentic_cs_3.csv", index=False)

In [18]:
cation_resin_3.describe()

,time_diff
count,747.000000
mean,45.909625
std,31.333074
min,1.530000
25%,27.185000
50%,40.330000
75%,56.235000
max,493.920000


从更换树脂当天至未来的某一天，当原树脂洗床所多消耗酸的成本可以覆盖更换树脂的成本时，就值得更换树脂。

### 4.制水量整合

In [19]:
import pandas as pd
import re
import calendar
import os
from time import sleep
from tqdm.auto import tqdm

# 初始化文件
if os.path.exists('WatarProductionRawInfo.csv'):
    os.remove('WatarProductionRawInfo.csv')
    
# 数据列
count_max_col = 8
count_min_col = 2
     
# 数据行    
valid_index1 = 3
valid_index2 = 4

years = [2019,2020,2021,2022]

wp_df = pd.DataFrame({'date':[], 'capacity':[]})
wp_df.to_csv("WatarProductionRawInfo.csv",mode='a',index=False)


# 遍历路径
for year in tqdm(years,desc="year"):


    # 遍历文件
    for month in tqdm(range(1,13,1),desc="month"):
        file_name = '干熄焦生产记录台账/{0}/干熄焦生产记录台账（{1}月）.xlsx'.format(year,month)

        # 返回（当月总周数，当月总天数）
        month_range = calendar.monthrange(int(year),int(month))
        
        # 遍历表单
        for day in range(1,month_range[1]+1,1):  # 从第1天开始至当月最后一天
            try:
                wp_sheet_df = pd.read_excel(file_name, sheet_name = "{}日".format(day), header=None) # 读取当日表单
            except FileNotFoundError as file_error:
                print(file_error)
                break
                
            # wp_sheet_df = pd.DataFrame(data=wp_sheet)    
              
            # 当日制水量
            occupy = wp_sheet_df[count_max_col][valid_index1] - wp_sheet_df[count_min_col][valid_index1]
            delivery = wp_sheet_df[count_max_col][valid_index2] - wp_sheet_df[count_min_col][valid_index2]
            daily_capacity_sum = occupy + delivery
            daily_capacity = {'capacity':[daily_capacity_sum]}
            daily_capacity_df = pd.DataFrame(data=daily_capacity)
            
            # 当日日期
            wp_date= {'date':["{0}-{1}-{2}".format(year, month, day)]} # 再生日期信息
            wp_date_df = pd.DataFrame(data=wp_date)
            
            # 追加入DataFrame
            wp_daily = pd.DataFrame([[wp_date_df, daily_capacity_df]], columns=['date', 'capacity'])
            
            # 直接插入csv文件
            wp_df = pd.concat([wp_date_df, daily_capacity_df], join="outer",axis=1, ignore_index=True)
            wp_df.to_csv("WatarProductionRawInfo.csv",mode='a',header=False, index=False)
            

year:   0%|          | 0/4 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

month:   0%|          | 0/12 [00:00<?, ?it/s]

[Errno 2] No such file or directory: '干熄焦生产记录台账/2022/干熄焦生产记录台账（12月）.xlsx'


### 2.制水量数据清洗

In [20]:
wp_data = pd.read_csv("WatarProductionRawInfo.csv")

# 得分异常值
wp_data_zscore = wp_data['capacity'].copy()
z_score = (wp_data_zscore - wp_data_zscore.mean()) / wp_data_zscore.std()
wp_data_zscore['expection'] = z_score.abs() > 2.2
wp_data_zscore.to_csv("test.csv", index=False)
# wp_data_score == True

In [21]:
wp_expection_data = wp_data.loc[(wp_data['capacity'] > 10000) | (wp_data['capacity'] < 0)]
wp_expection_data = wp_expection_data.copy()
wp_expection_data.sort_values("date", ascending=False, inplace=True)
wp_expection_data

,date,capacity
1401,2022-11-2,-80.0
946,2021-8-4,-7320.0
968,2021-8-26,-138450.0
833,2021-4-13,451013.0
777,2021-2-16,267140.0
1018,2021-10-15,-78280.0
755,2021-1-25,10436.0
748,2021-1-18,45972.0
630,2020-9-22,10215.0
583,2020-8-6,-9725.0
